In [2]:
import os
import json

In [21]:
from doccano_transformer.datasets import NERDataset
from doccano_transformer.utils import read_jsonl
from tqdm import tqdm

In [25]:
dataset = read_jsonl(filepath='all.jsonl', dataset=NERDataset, encoding='utf-8')
conll = dataset.to_conll2003(tokenizer=str.split)
with open('dataset.conll', 'w') as f:
    for item in tqdm(conll):
        f.write(item['data'])

1086it [00:00, 10934.68it/s]


In [26]:
import re
chars = [',', '.', '?', '!', '"', ':', '(', ')', '/']
with open('dataset_pure_punct.conll', 'w') as output_file:
    with open('dataset.conll', 'r') as input_file:
        first = True
        for line in input_file:
            if line.startswith('-DOCSTART-'):
                if not first:
                    output_file.write('\n')
                else:
                    first = False
            elif line == '\n':
                pass
            else:
                contents = line.split()
                word = contents[0]
                tag = contents[3]
                texts = list(filter(None, re.split('([,|.|?|!|"|:|(|)|/])', word)))
                end_index = len(texts)
                for text in texts[::-1]:
                    if text in chars:
                        end_index -= 1
                    else:
                        break
                for idx, text in enumerate(texts):
                    temp_tag = tag
                    if idx > 0 and tag.startswith('B-'):
                        temp_tag = tag.replace('B-', 'I-')
                    if idx >= end_index:
                        temp_tag = 'O'
                    output_file.write("{} {}\n".format(text, temp_tag))

In [1]:
import random
proportion = 80

with open('dataset_pure_punct.conll', 'r') as input_file:
    with open('train.txt', 'w') as train_file:
        with open('dev.txt', 'w') as dev_file:
            cache = []
            for line in input_file:
                if line != '\n':
                    cache.append(line)
                else:
                    dice = random.randint(1, 100)
                    if dice > proportion:
                        for item in cache:
                            dev_file.write(item)
                        dev_file.write('\n')
                    else:
                        for item in cache:
                            train_file.write(item)
                        train_file.write('\n')
                    cache.clear()
            if len(cache) > 0:
                dice = random.randint(1, 100)
                if dice > proportion:
                    for item in cache:
                        dev_file.write(item)
                    dev_file.write('\n')
                else:
                    for item in cache:
                        train_file.write(item)
                    train_file.write('\n')